## Tutorial #1: Loading Data

#### This tutorial focuses on loading EEG data in several different ways.  

In [6]:
import mne #open-source Python package for exploring, visualizing, and analyzing human neurophysiological data

EEG data can be kept in 3 different types: raw data, epoched data and evoked (averaged) data.

1. Raw Data: Continues data is stored in raw object in MNE. A raw object has 2D array structure as channels×samples.
   
2. Epoched Data: Collection of time-locked trials. It is stored as events×channles×times.

3. Evoked Data: When epoched data is averaged out over trials, evoked data is produced. The output is time-locked to the event. It is stored as channels×times.

#### 1.1) Loading datasets available in MNE: EEGBCI motor imagery Dataset


In [23]:
import os
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
# The parameters for loading data are as following: 
# subject(s) whose data will be fetched, runs corresponding to different tasks, path to locate fetched data
# load_data() function returns name of the files that the data is kept.

#Define the parameters 
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # use only hand and feet motor imagery runs

#Get data and locate in to given path
files = eegbci.load_data(subject, runs, '../datasets/')
#Read raw data files where each file contains a run
raws = [read_raw_edf(f, preload=True) for f in files]
#Combine all loaded runs
raw_data = concatenate_raws(raws)

Extracting EDF parameters from /Users/scg/Desktop/AlgorithmsNeuroscience/datasets/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/scg/Desktop/AlgorithmsNeuroscience/datasets/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/scg/Desktop/AlgorithmsNeuroscience/datasets/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [44]:
#The following line can be useful for getting general information about the data
raw_data.info

<Info | 16 non-empty fields
    bads : list | 0 items
    ch_names : list | Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., ...
    chs : list | 64 items (EEG: 64)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 80.0 Hz
    meas_date : tuple | 2009-08-12 16:15:00 GMT
    nchan : int | 64
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 160.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xplotter

In [34]:
#Extract events from raw data
events, event_ids = mne.events_from_annotations(raw_data, event_id='auto')
event_ids

Used Annotations descriptions: ['T0', 'T1', 'T2']


{'T0': 1, 'T1': 2, 'T2': 3}

In [40]:
tmin, tmax = -1, 4  # define epochs around events (in s)
#event_ids = dict(hands=2, feet=3)  # map event IDs to tasks

epochs = mne.Epochs(raw_data, events, event_ids, tmin - 0.5, tmax + 0.5, baseline=None, preload=True)

90 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 90 events and 961 original time points ...
3 bad epochs dropped


In [42]:
#Access to the data
epochs._data

array([[[ 1.40e-05, -1.00e-05,  5.00e-06, ..., -9.70e-05, -1.10e-04,
         -8.30e-05],
        [ 2.70e-05,  8.00e-06,  2.10e-05, ..., -7.70e-05, -9.00e-05,
         -6.10e-05],
        [ 3.10e-05,  1.40e-05,  2.20e-05, ..., -4.50e-05, -5.90e-05,
         -3.90e-05],
        ...,
        [ 5.30e-05,  5.40e-05,  7.10e-05, ..., -3.20e-05, -2.00e-05,
         -2.40e-05],
        [ 2.90e-05,  4.00e-05,  6.90e-05, ..., -2.30e-05, -6.00e-06,
         -7.00e-06],
        [-3.30e-05, -3.10e-05, -7.00e-06, ..., -1.60e-05,  1.00e-06,
         -1.10e-05]],

       [[-1.20e-05,  2.00e-06,  3.20e-05, ...,  7.50e-05,  4.20e-05,
          3.20e-05],
        [-6.00e-06,  4.00e-06,  3.00e-05, ...,  9.10e-05,  5.20e-05,
          3.90e-05],
        [-2.00e-06,  1.00e-06,  2.50e-05, ...,  1.18e-04,  8.10e-05,
          6.30e-05],
        ...,
        [-1.10e-05,  1.00e-06,  1.40e-05, ...,  6.20e-05,  5.50e-05,
          3.90e-05],
        [-9.00e-06, -3.00e-06,  7.00e-06, ...,  7.90e-05,  6.30e-05,
   

#### 1.2) Loading data that is not available in MNE

In [48]:
## Loading Epoched data 

# The file name should end with'epo' to be able to load it via mne
data_file = '../../817_1_PDDys_ODDBALL_Clean_curated-epo'

# Read the EEG epochs:
epochs = mne.read_epochs(data_file + '.fif')
print(len(epochs))

Reading ../../817_1_PDDys_ODDBALL_Clean_curated-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
189


In [46]:
#The following line can be useful for getting general information about the data
epochs.info

<Info | 19 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fz, F3, F7, FC5, FC1, C3, T7, CP5, ...
    chs : list | 60 items (EEG: 60)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 60 items (60 EEG)
    events : list | 0 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 250.0 Hz
    meas_date : NoneType | unspecified
    meas_id : dict | 4 items
    nchan : int | 60
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xp

In [50]:
## Accessing to the data

# The data can be accessed via:
epochs_data_1 = epochs._data
#or 
epochs_data_2 = epochs.get_data()
#To check whether the two different ways returns the same data
if epochs_data_1.all() == epochs_data_2.all():
    print("Same data!")

Same data!


In [51]:
# To get names of all the conditions in given data and number of events in each condition
epochs.event_id

{'Standard': 201, 'Target': 200, 'Novel': 202}

In [52]:
#To access epochs with a specific condition
novels = epochs['Novel']

In [53]:
tmin = epochs.tmin
tmax = epochs.tmax

print('Start time before the event' , tmin)
print('Stop time after the event' , tmax)

Start time before the event -0.1
Stop time after the event 0.5
